In [ ]:
import os
import sys
os.chdir('/home/deep09/project_vlad/parallel-tempering/')
import tensorflow as tf
import numpy as np
from simulator import read_datasets
from simulator.simulator import Simulator
from simulator.models.cifar10_models_v2 import lenet5_with_dropout
#from simulator.models.emnist_models import lenet5_with_lr
#from simulator.models.emnist_models import lenet5_with_input_noise
#from simulator.models.emnist_models import lenet5 # <-- for regularization or weight_noise
#from simulator.models.emnist_models import lenet5_with_const_dropout
from simulator import simulator_utils as s_utils
import gc




In [ ]:
# dropout proba_coeff higher ==> accept higher
params = dict(
        n_epochs=[251, 251, 2000, 2000, 2000, 2000],
        learning_rate=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
        n_replicas=[8, 8, 8, 8, 10, 10, 10],
        noise_type=['dropout_gd', 'dropout_gd', 'weight_noise', 'weight_noise', 'input_noise', 'l2_regularizer'],
        func_name=['lenet5', 'lenet5', 'lenet5', 'lenet5', 'lenet5', 'lenet5', 'lenet5', 'lenet5'],
        dataset_name=['cifar', 'cifar', 'emnist', 'emnist', 'emnist', 'emnist'],
        loss_func_name=['cross_entropy', 'cross_entropy', 'cross_entropy', 'cross_entropy', 'cross_entropy', 'cross_entropy'],
        train_data_size=[45000, 45000, 79920, 79920, 79920, 79920],
        model=[lenet5_with_dropout, lenet5_with_dropout, ],
        n_simulations=[1, 1, 1, 1, 1, 1],
        description=['test', 'test', 'test', 'test', 'test', 'test'],
        separation_ratio=[0, 0, 0, 0, 0, 0],
        beta_0=[.52, .52, 0.0001, 0.0001, 0.05, 0.01, ],
        beta_n=[.65,  0.65, 0.00001, 0.00001, 0.0000001, 0.0000001],
        proba_coeff=[10, 0.5, 1, 1, 1, 1, ],
        burn_in_period=[12000, np.inf, np.inf, np.inf, np.inf, np.inf],
        batch_size=[128, 128, 128, 128, 128, 128],
        swap_step=[100, 2000, 2000, 2000, 2000, 2000, 2000,],
        mode=[None, None, None, None, None, None]
        )

test_step = 352
n_experiments = 2
timer = s_utils.Timer()
timer.start_timer()
total_sims = np.sum(params['n_simulations'])

In [ ]:
x_train, y_train, x_test, y_test, x_valid, y_valid = read_datasets._create_cifar_data_or_get_existing_lenet5()
for i in range(n_experiments):
    #if i ==0: continue
    simulation_num = 0
    noise_list = np.linspace(params['beta_0'][i],
                             params['beta_n'][i],
                             params['n_replicas'][i]) 
    noise_list = list(noise_list)
    ###############################################

    ###############################################
    
    name = s_utils.generate_experiment_name(model_name=params['func_name'][i],
                                            dataset_name=params['dataset_name'][i],
                                            separation_ratio=params['separation_ratio'][i],
                                            n_replicas=params['n_replicas'][i],
                                            beta_0=params['beta_0'][i],
                                            beta_n=params['beta_n'][i],
                                            loss_func_name=params['loss_func_name'][i],
                                            swap_step=params['swap_step'][i],
                                            burn_in_period=params['burn_in_period'][i],
                                            learning_rate=params['learning_rate'][i],
                                            n_epochs=params['n_epochs'][i],
                                            noise_type=params['noise_type'][i],
                                            batch_size=params['batch_size'][i],
                                            proba_coeff=params['proba_coeff'][i],
                                            train_data_size=params['train_data_size'][i],
                                            mode=params['mode'][i],
                                            version='v4')
    
    
    #proba_coeffs = list(np.linspace(100000, 50000, params['n_replicas'][i] -1))
    #proba_coeffs = [2000, 5000, 6000, 100000, 100000]
    #proba_coeffs = s_utils.generate_proba_coeffs(noise_list, mult=1000)
    #print(proba_coeffs)
    scheduled_lrs = {1:0.1, 12000:0.01}
    
    proba_coeffs = params['proba_coeff'][i]
    #scheduled_lrs = None
    sim = Simulator(model=params['model'][i],
                    learning_rate=params['learning_rate'][i],
                    noise_list=noise_list,
                    noise_type=params['noise_type'][i],
                    batch_size=params['batch_size'][i],
                    n_epochs=params['n_epochs'][i],
                    name=name,
                    burn_in_period=params['burn_in_period'][i],
                    scheduled_lr=scheduled_lrs,
                    swap_step=params['swap_step'][i],
                    separation_ratio=params['separation_ratio'][i],
                    n_simulations=params['n_simulations'][i],
                    test_step=test_step,
                    loss_func_name=params['loss_func_name'][i],
                    #proba_coeff=params['proba_coeff'][i],
                    proba_coeff=proba_coeffs,
                    mode=params['mode'][i])
    
    for sim_num in range(params['n_simulations'][i]):
        simulation_num += 1
        '''
        train_data, train_labels, test_data, test_labels, valid_data, valid_labels = (
                                    read_datasets.get_cifar10_data())
        
        '''
        
        
        print(name, str(i) + '/' + str(n_experiments))
        
        print('train-data-shape: {}'.format(x_train.shape))
        print('test-data-shape: {}'.format(x_test.shape))
        print('valid-data-shape: {}'.format(x_valid.shape))
        print('noise_list:')
        print(["{0:.6f}".format(n) for n in sorted(noise_list)])
        sim.train(train_data_size=None,
                  train_data=x_train,
                  train_labels=y_train,
                  validation_data=x_valid,
                  validation_labels=y_valid,
                  test_data=x_test,
                  test_labels=y_test)
        del sim
        gc.collect()
        print()
        print(str(simulation_num) + '/' + str(n_experiments), ', time took:', timer.elapsed_time())